In [19]:
import pandas as pd
from konlpy.tag import Okt
from konlpy.tag import Mecab
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from gensim import corpora
import gensim
import pyLDAvis.gensim
import pyLDAvis
import pyLDAvis.sklearn
import re

In [20]:
df = pd.read_excel('naver_shopping_data.xlsx', )

In [21]:
df.drop('Unnamed: 0', axis = 1, inplace = True)

In [22]:
# 1. 데이터 전처리
okt = Okt()
stopwords = ['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '를', '으로', '자', '에', '와', '한', '하다', '너무']

def preprocess(text):
    text = re.sub('[^가-힣]', ' ', text)  # 한글 외 문자 제거
    tokenized = okt.morphs(text, stem=True)  # 토큰화
    tokenized = [word for word in tokenized if word not in stopwords and len(word) > 1]  # 불용어 제거
    return tokenized

df['processed_review'] = df['review'].apply(preprocess)

In [23]:
# 2. 감성 분류 (별점으로 나눔)
df['sentiment'] = df['rating'].apply(lambda x: 'positive' if x >= 3 else 'negative')

In [24]:
# 4. 제품별, 매장별, 감성별로 리뷰 그룹화
grouped = df.groupby(['pdt_name', 'store', 'sentiment'])

lda 모델링
1) gensim 라이브러리: 토픽 모델링, 자연어 처리에 특화된 라이브러리
-> 
dictionary 생성(문서 내 모든 유니크한 단어에 대한 사전 생성) 
& 
corpus 생성 (각 문서를 단어 id, 단어 빈도수 형태로 변환: gensim의 'bag of words' 모델 사용)

2)  scikit-learn 라이브러리: 널리 사용되는 머신 러닝 라이브러리
->
countvectorizer 사용 ('단어 빈도수' 행렬로 변환: 각 단어가 문서에서 몇 번 나타났는지 나타냄)
&
hyperparemeter 설정: 'n_components'(토픽수), 'random_state' 등

차이점
1) 라이브러리 종류:
gensim(자연어 처리에 특화) vs scikit-learn(일반적 머신러닝에 적합)

2) 데이터 처리 방식:
gensim(단어id, 빈도수 형태의 corpus 사용) vs scikit-learn(단어 빈도수 행렬 학습)

3) 모델 학습:
라이브러리에 따라 lda 모델 학습 방식에 차이 있음

In [25]:
def lda_visualize(model, corpus, dictionary, rating):
    # LDA 시각화 준비
    pyLDAvis.enable_notebook()
    vis_data = pyLDAvis.gensim.prepare(model, corpus, dictionary, sort_topics=False)
    pyLDAvis.display(vis_data)
    # HTML 파일로 저장
    html_file_path = f'./lda_visualization_{store}_{sentiment}.html'

    pyLDAvis.save_html(vis_data, html_file_path)

    return vis_data

In [26]:
for (pdt_name, store, sentiment), group in grouped:
    reviews = group['processed_review'].tolist() # 리뷰 데이터 추출
    
    if not reviews:
        continue 

    # LDA 토픽 모델링
    dictionary = corpora.Dictionary(reviews) # gensim 라이브러리 사용: 각 단어에 대한 unique ID 할당하는 사전 생성
    
    if len(dictionary) == 0:
        continue
    
    corpus = [dictionary.doc2bow(text) for text in reviews] # 각 리뷰를 (단어ID, 단어 빈도수) 형태의 튜플로 변환: bag of words 모델 나타냄

    if not any(corpus):
        continue
    
    lda = gensim.models.ldamodel.LdaModel(corpus=corpus, num_topics=2, id2word=dictionary) # gensim 라이브러리의 lda 모델 사용: 토픽 모델링 수행
    
    # 토픽 출력: 학습된 lda 모델에서 각 토픽 & 그 토픽 관련 주요 단어 출력
    # idx: 토픽 번호, topic: 해당 토픽의 주요 단어 & 그 단어의 관련성 가중치
    print(f'제품: {pdt_name}, 매장: {store}, 감성: {sentiment}')
    for idx, topic in lda.print_topics(-1): # print_topics 메소드: 각 토픽의 주요 단어와 그 단어들의 관련성 가중치 출력
        print('Topic: {} \nWords: {}'.format(idx, topic))
    print("\n")



제품: GGPX 폭스퍼트리밍퀼팅롱패딩덕다운점퍼 GMDBDU993F, 매장: CJ온스타일, 감성: negative
Topic: 0 
Words: 0.118*"만큼" + 0.117*"부다" + 0.117*"눈밭" + 0.114*"어도" + 0.111*"뒹굴" + 0.110*"좋다" + 0.109*"되다" + 0.104*"따뜻하다" + 0.100*"보이다"
Topic: 1 
Words: 0.126*"보이다" + 0.121*"따뜻하다" + 0.114*"되다" + 0.113*"좋다" + 0.111*"뒹굴" + 0.107*"어도" + 0.104*"눈밭" + 0.103*"부다" + 0.101*"만큼"


제품: GGPX 폭스퍼트리밍퀼팅롱패딩덕다운점퍼 GMDBDU993F, 매장: GGPX, 감성: negative
Topic: 0 
Words: 0.150*"가격" + 0.150*"구입" + 0.145*"기분" + 0.144*"안좋다" + 0.140*"내다" + 0.139*"바로" + 0.133*"없다"
Topic: 1 
Words: 0.160*"없다" + 0.149*"바로" + 0.148*"내다" + 0.142*"안좋다" + 0.140*"기분" + 0.131*"구입" + 0.131*"가격"


제품: GGPX 폭스퍼트리밍퀼팅롱패딩덕다운점퍼 GMDBDU993F, 매장: GGPX, 감성: positive
Topic: 0 
Words: 0.031*"가볍다" + 0.027*"입다" + 0.023*"좋다" + 0.019*"편하다" + 0.017*"들다" + 0.015*"사이즈" + 0.013*"따뜻하다" + 0.012*"길이" + 0.012*"크다" + 0.011*"조금"
Topic: 1 
Words: 0.023*"좋다" + 0.023*"자다" + 0.023*"입다" + 0.021*"가볍다" + 0.013*"이쁘다" + 0.012*"패딩" + 0.011*"않다" + 0.011*"있다" + 0.011*"같다" + 0.011*"사다"


제품: GGPX 폭스퍼트리밍퀼팅롱패딩덕다운점퍼 GM